# Discharge Notes and Readmission Rates: Modeling

This notebook is for the modeling the cleaned dataset.

## Import

In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re